# **CREATING A FACT TABLE**

## Reading the silver data

In [1]:
df_src=spark.sql('''
    SELECT * FROM PARQUET.`abfss://incre_ws@onelake.dfs.fabric.microsoft.com/lake_san.Lakehouse/Files/silver/carsales`
    ''')
display(df_src)

StatementMeta(, 605145f7-e019-4386-a02b-3bab283a82af, 3, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, b08bbcf8-417f-47a3-a7f2-355ea01f590a)

## Reading the dimensions

In [2]:
df_model=spark.sql("SELECT * FROM dim_model")
df_branch=spark.sql("SELECT * FROM dim_branch")
df_dealer=spark.sql("SELECT * FROM dim_dealer")
df_date=spark.sql("SELECT * FROM dim_date")

StatementMeta(, 605145f7-e019-4386-a02b-3bab283a82af, 4, Finished, Available, Finished)

## Joining

In [3]:
df_fact=df_src.join(df_branch,df_src['Branch_ID']==df_branch['Branch_ID'],'left')\
                .join(df_dealer,df_src['Dealer_ID']==df_dealer['Dealer_ID'],'left')\
                .join(df_model,df_src['Model_ID']==df_model['Model_ID'],'left')\
                .join(df_date,df_src['Date_ID']==df_date['Date_ID'],'left')\
                .select(df_src['Revenue'],df_src['Units_sold'],df_src['Revenue_perUnit'],df_branch['dim_Branch_key'],df_dealer['dim_Dealer_key'],df_model['dim_model_key'],df_date['dim_date_key'])

StatementMeta(, 605145f7-e019-4386-a02b-3bab283a82af, 5, Finished, Available, Finished)

In [4]:
df_fact = df_fact.dropDuplicates(['dim_model_key', 'dim_Branch_key', 'dim_date_key', 'dim_Dealer_key'])
display(df_fact)

StatementMeta(, 605145f7-e019-4386-a02b-3bab283a82af, 6, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, e8709a56-c4a8-4614-8d33-2ea06f42bb90)

## Writing fact table

In [5]:
from delta.tables import DeltaTable

StatementMeta(, 605145f7-e019-4386-a02b-3bab283a82af, 7, Finished, Available, Finished)

In [6]:
if spark.catalog.tableExists('fact_sales'):
    deltatable=DeltaTable.forPath(spark,'abfss://incre_ws@onelake.dfs.fabric.microsoft.com/lake_san.Lakehouse/Files/gold/Fact_sales')
    deltatable.alias("trg").merge(df_fact.alias("src"),'trg.dim_model_key=src.dim_model_key and trg.dim_Branch_key=src.dim_Branch_key and trg.dim_date_key=src.dim_date_key and trg.dim_Dealer_key=src.dim_Dealer_key')\
                            .whenMatchedUpdateAll()\
                            .whenNotMatchedInsertAll()\
                            .execute()
else:
    df_fact.write.format('delta')\
                    .mode('overwrite')\
                    .option('path','abfss://incre_ws@onelake.dfs.fabric.microsoft.com/lake_san.Lakehouse/Files/gold/Fact_sales')\
                    .saveAsTable('fact_sales')

StatementMeta(, 605145f7-e019-4386-a02b-3bab283a82af, 8, Finished, Available, Finished)

In [7]:
%%sql
SELECT * FROM fact_sales

StatementMeta(, 605145f7-e019-4386-a02b-3bab283a82af, 9, Finished, Available, Finished)

<Spark SQL result set with 1000 rows and 7 fields>

In [8]:
%%sql
SELECT COUNT(*) FROM fact_sales

StatementMeta(, 605145f7-e019-4386-a02b-3bab283a82af, 10, Finished, Available, Finished)

<Spark SQL result set with 1 rows and 1 fields>